In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [7]:
input_x=np.random.random((1,3))

In [8]:
print(input_x)

[[ 0.04781787  0.35512365  0.89200966]]


In [15]:
wts_w=np.random.random((3,1));

In [16]:
print(wts_w)

[[ 0.6566561 ]
 [ 0.44513859]
 [ 0.01614509]]


In [17]:
print(input_x*wts_w)

[[ 0.03139989  0.23319411  0.58574359]
 [ 0.02128558  0.15807924  0.39706792]
 [ 0.00077202  0.0057335   0.01440157]]


In [18]:
np.dot(input_x,wts_w)

array([[ 0.20388071]])